# Reverse Sources

Created: 2019.11.11  
Notebook sequence: 12

Finding the predictions of lean from that one place that did it based on facebook follower demographics

---

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import util

labels_df = util.nela_load_labels()
mbm_data = util.nela_load_media_bias_monitor()

# Counts

In [10]:
count = 0
bias = [0, 0, 0, 0, 0]
for src in mbm_data:
    value = mbm_data[src]
    if value: 
        count += 1
        if value < -1.2: 
            bias[0] += 1
        elif value < -.4:
            bias[1] += 1
        elif value < .4:
            bias[2] += 1
        elif value < 1.2:
            bias[3] += 1
        else:
            bias[4] += 1
print(count)
print(bias)

82
[5, 35, 23, 4, 15]


In [12]:
mbfc_lbl = "Media Bias / Fact Check, label"
mbfc_bias = [           
    len(list(labels_df[(labels_df[mbfc_lbl] == "left_bias")].Source)),
    len(list(labels_df[(labels_df[mbfc_lbl] == "left_center_bias")].Source)),
    len(list(labels_df[(labels_df[mbfc_lbl] == "least_biased")].Source)),
    len(list(labels_df[(labels_df[mbfc_lbl] == "right_center_bias")].Source)),
    len(list(labels_df[(labels_df[mbfc_lbl] == "right_bias")].Source)),
]
print(mbfc_bias)

[30, 36, 3, 10, 16]


In [13]:
as_lbl = "Allsides, bias_rating"
as_bias = [
    len(list(labels_df[(labels_df[as_lbl] == "Left")].Source)),
    len(list(labels_df[(labels_df[as_lbl] == "Lean Left")].Source)),
    len(list(labels_df[(labels_df[as_lbl] == "Center")].Source)),
    len(list(labels_df[(labels_df[as_lbl] == "Right")].Source)),
    len(list(labels_df[(labels_df[as_lbl] == "Lean Right")].Source)),
]
print(as_bias)

[17, 13, 11, 14, 10]


# Lists

In [31]:
import copy

In [32]:
mbm_sources = {
    "left": [],
    "left-center": [],
    "center": [],
    "right-center": [],
    "right": []
}
alt_mbm_sources = {
    "left": [],
    "left-center": [],
    "center": [],
    "right-center": [],
    "right": []
}

for src in mbm_data:
    value = mbm_data[src]
    if value: 
        if value < -1.2: 
            mbm_sources["left"].append(src)
        elif value < -.4:
            mbm_sources["left-center"].append(src)
        elif value < .4:
            mbm_sources["center"].append(src)
        elif value < 1.2:
            mbm_sources["right-center"].append(src)
        else:
            mbm_sources["right"].append(src)
            
        if value < -.67: 
            alt_mbm_sources["left"].append(src)
        elif value < .67:
            alt_mbm_sources["center"].append(src)
        else:
            alt_mbm_sources["right"].append(src)

mbfc_sources = {
    "left": list(labels_df[(labels_df[mbfc_lbl] == "left_bias")].Source),
    "left-center": list(labels_df[(labels_df[mbfc_lbl] == "left_center_bias")].Source),
    "center": list(labels_df[(labels_df[mbfc_lbl] == "least_biased")].Source),
    "right-center": list(labels_df[(labels_df[mbfc_lbl] == "right_center_bias")].Source),
    "right": list(labels_df[(labels_df[mbfc_lbl] == "right_bias")].Source)
}
alt_mbfc_sources = {
    "left": [],
    "left-center": [],
    "center": [],
    "right-center": [],
    "right": []
}
alt_mbfc_sources["left"] = copy.deepcopy(mbfc_sources["left"])
alt_mbfc_sources["left"].extend(copy.deepcopy(mbfc_sources["left-center"]))
alt_mbfc_sources["center"] = copy.deepcopy(mbfc_sources["center"])
alt_mbfc_sources["right"] = copy.deepcopy(mbfc_sources["right"])
alt_mbfc_sources["right"].extend(copy.deepcopy(mbfc_sources["right-center"]))

as_sources = {
    "left": list(labels_df[(labels_df[as_lbl] == "Left")].Source),
    "left-center": list(labels_df[(labels_df[as_lbl] == "Lean Left")].Source),
    "center": list(labels_df[(labels_df[as_lbl] == "Center")].Source),
    "right-center": list(labels_df[(labels_df[as_lbl] == "Right")].Source),
    "right": list(labels_df[(labels_df[as_lbl] == "Lean Right")].Source)
}
alt_as_sources = {
    "left": [],
    "left-center": [],
    "center": [],
    "right-center": [],
    "right": []
}
alt_as_sources["left"] = copy.deepcopy(as_sources["left"])
alt_as_sources["left"].extend(copy.deepcopy(as_sources["left-center"]))
alt_as_sources["center"] = copy.deepcopy(as_sources["center"])
alt_as_sources["right"] = copy.deepcopy(as_sources["right"])
alt_as_sources["right"].extend(copy.deepcopy(as_sources["right-center"]))

In [33]:
unique = {
    "left": [],
    "left-center": [],
    "center": [],
    "right-center": [],
    "right": []
}

votes = {
    "left": {},
    "left-center": {},
    "center": {},
    "right-center": {},
    "right": {}
}

def tally_votes(key, sources):
    global votes
    
    for src in sources[key]:
        if src not in votes[key]:
            votes[key][src] = 1
        else:
            votes[key][src] += 1
            
mbm_use = alt_mbm_sources
mbfc_use = alt_mbfc_sources
as_use = alt_as_sources

for key in ["left", "left-center", "center", "right-center", "right"]:
    print("-----",key,"-----")
    print(mbm_use[key])
    print(mbfc_use[key])
    print(as_use[key])
    print("\n\n")
    
    combined = []
    combined.extend(mbm_use[key])
    combined.extend(mbfc_use[key])
    combined.extend(as_use[key])
    combined = list(set(combined))
    
    unique[key] = combined
    
    tally_votes(key, mbm_use)
    tally_votes(key, mbfc_use)
    tally_votes(key, as_use)

----- left -----
['Addicting Info', 'Al Jazeera', 'Alternet', 'Daily Beast', 'Daily Kos', 'Faking News', 'Feministing Blog', 'FiveThirtyEight', 'France24', 'MSNBC', 'Mercury News', 'MotherJones', 'NPR', 'New Yorker', 'Pink News UK', 'Politico', 'Raw story', 'Slate', 'Spiegel', 'Sputnik', 'Talking Points Memo', 'The Atlantic', 'ThinkProgress', 'Vox']
['Addicting Info', 'Alternet', 'CNN', 'Crooks and Liars', 'Daily Beast', 'Daily Kos', 'Feministing Blog', 'Forward Progessives', 'MSNBC', 'Media Matters for America', 'MotherJones', 'New Yorker', 'Newsweek', 'Palmer Report', 'Pink News UK', 'Pravada Report', 'Raw Story', 'RightWingWatch', 'Salon', 'Shadow Proof', 'Shareblue', 'Slate', 'Talking Points Memo', 'Telesur TV', 'The Daily Mirror', 'The Daily Record', 'The Huffington Post', 'ThinkProgress', 'Vox', 'Washington Monthly', 'ABC News', 'Al Jazeera', 'BBC', 'Birmingham Mail', 'Business Insider', 'Buzzfeed', 'CBS News', 'CNBC', 'Chicago Sun-Times', 'Democracy 21', 'FiveThirtyEight', 'Fran

In [34]:
for key in votes.keys():
    print("-----",key,"-----")
    for src in votes[key]:
        print(src,votes[key][src])
        
    print("\n\n")

----- left -----
Addicting Info 2
Al Jazeera 2
Alternet 3
Daily Beast 3
Daily Kos 3
Faking News 1
Feministing Blog 2
FiveThirtyEight 2
France24 2
MSNBC 3
Mercury News 2
MotherJones 3
NPR 2
New Yorker 3
Pink News UK 2
Politico 1
Raw story 1
Slate 3
Spiegel 2
Sputnik 1
Talking Points Memo 2
The Atlantic 3
ThinkProgress 3
Vox 3
CNN 2
Crooks and Liars 1
Forward Progessives 1
Media Matters for America 2
Newsweek 2
Palmer Report 1
Pravada Report 1
Raw Story 2
RightWingWatch 1
Salon 2
Shadow Proof 1
Shareblue 1
Telesur TV 1
The Daily Mirror 1
The Daily Record 1
The Huffington Post 2
Washington Monthly 2
ABC News 2
BBC 1
Birmingham Mail 1
Business Insider 1
Buzzfeed 2
CBS News 2
CNBC 1
Chicago Sun-Times 2
Democracy 21 1
Fusion 1
Interpreter Mag 1
New York Daily News 2
PBS 1
SkyNewsPolitics 1
SkyNewsUS 1
The Denver Post 1
The Guardian 2
The Hill 1
The Independent 1
The Intercept 2
The Irish Times 1
The Moscow Times 1
The New York Times 2
The Verge 2
USA Today 1
Washington Post 2
Wings Over Scot

In [40]:
def find_src(src, sources, name):
    for key in sources.keys():
        if src in sources[key]:
            print("\tFound",src,"for",name,"in",key)

for src in mbm_sources["center"]:
    print(src, mbm_data[src])
    
    find_src(src, mbfc_sources, "mbfc")
    find_src(src, as_sources, "as")

ABC News -0.117
	Found ABC News for mbfc in left-center
	Found ABC News for as in left-center
BBC UK -0.241
Birgmingham Mail 0.156
Business Insider -0.215
	Found Business Insider for mbfc in left-center
	Found Business Insider for as in center
Buzzfeed -0.268
	Found Buzzfeed for mbfc in left-center
	Found Buzzfeed for as in left-center
CBS News 0.244
	Found CBS News for mbfc in left-center
	Found CBS News for as in left-center
CNBC -0.075
	Found CNBC for mbfc in left-center
	Found CNBC for as in center
CNN -0.342
	Found CNN for mbfc in left
	Found CNN for as in left-center
Chicago Sun-Times -0.204
	Found Chicago Sun-Times for mbfc in left-center
	Found Chicago Sun-Times for as in left-center
Common Dreams -0.035
Daily Mail -0.021
	Found Daily Mail for as in right-center
Foreign Policy -0.174
	Found Foreign Policy for mbfc in center
New York Daily News -0.359
	Found New York Daily News for mbfc in left-center
	Found New York Daily News for as in left
New York Post -0.138
	Found New York

In [42]:
for src in as_sources["center"]:
    print(src)
    
    find_src(src, mbm_sources, "mbm")
    find_src(src, mbfc_sources, "mbfc")

Al Jazeera
	Found Al Jazeera for mbm in left-center
	Found Al Jazeera for mbfc in left-center
BBC
	Found BBC for mbm in left-center
	Found BBC for mbfc in left-center
Business Insider
	Found Business Insider for mbm in center
	Found Business Insider for mbfc in left-center
CNBC
	Found CNBC for mbm in center
	Found CNBC for mbfc in left-center
FiveThirtyEight
	Found FiveThirtyEight for mbm in left-center
	Found FiveThirtyEight for mbfc in left-center
NPR
	Found NPR for mbm in left-center
	Found NPR for mbfc in left-center
PBS
	Found PBS for mbm in left-center
	Found PBS for mbfc in left-center
Real Clear Politics
	Found Real Clear Politics for mbm in center
	Found Real Clear Politics for mbfc in right-center
Reuters
	Found Reuters for mbm in left-center
	Found Reuters for mbfc in center
The Hill
	Found The Hill for mbm in left-center
	Found The Hill for mbfc in left-center
USA Today
	Found USA Today for mbm in center
	Found USA Today for mbfc in left-center


In [43]:
for src in mbfc_sources["center"]:
    print(src)
    
    find_src(src, mbm_sources, "mbm")
    find_src(src, as_sources, "as")

Foreign Policy
	Found Foreign Policy for mbm in center
Politico
	Found Politico for mbm in left-center
Reuters
	Found Reuters for mbm in left-center
	Found Reuters for as in center
